In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd


from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import load_dataset

from sklearn.metrics import confusion_matrix, classification_report

In [3]:
emotions_data  = load_dataset("emotion")

emotions_data

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
emotions_data["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [5]:
emotions_data["train"][:5]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy'],
 'label': [0, 0, 3, 2, 3]}

In [6]:
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def tokenize_function(example):
  return tokenizer(example["text"], truncation=True)

tokenized_datasets = emotions_data.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [8]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [12]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

loss = SparseCategoricalCrossentropy(from_logits=True)

num_epochs = 10
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)


opt = Adam(learning_rate=lr_scheduler)


print(num_train_steps)

20000


In [13]:
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
model.fit(tf_train_dataset, epochs=10, batch_size=8, validation_data=tf_validation_dataset)

Epoch 1/10
2000/2000 [==============================] - 310s 131ms/step - loss: 0.3679 - accuracy: 0.8724 - val_loss: 0.1668 - val_accuracy: 0.9265
Epoch 2/10
2000/2000 [==============================] - 256s 128ms/step - loss: 0.1492 - accuracy: 0.9370 - val_loss: 0.1775 - val_accuracy: 0.9310
Epoch 3/10
2000/2000 [==============================] - 248s 124ms/step - loss: 0.1154 - accuracy: 0.9432 - val_loss: 0.1274 - val_accuracy: 0.9365
Epoch 4/10
2000/2000 [==============================] - 247s 124ms/step - loss: 0.0969 - accuracy: 0.9518 - val_loss: 0.1369 - val_accuracy: 0.9380
Epoch 5/10
2000/2000 [==============================] - 248s 124ms/step - loss: 0.0795 - accuracy: 0.9598 - val_loss: 0.1279 - val_accuracy: 0.9430
Epoch 6/10
2000/2000 [==============================] - 247s 123ms/step - loss: 0.0630 - accuracy: 0.9695 - val_loss: 0.1850 - val_accuracy: 0.9355
Epoch 7/10
2000/2000 [==============================] - 247s 124ms/step - loss: 0.0435 - accuracy: 0.9818 - val_

In [14]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  4614      
                                                                 
Total params: 109486854 (417.66 MB)
Trainable params: 109486854 (417.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.evaluate(tf_validation_dataset)

250/250 [==============================] - 12s 46ms/step - loss: 0.2493 - accuracy: 0.9385


[0.24932052195072174, 0.9384999871253967]

In [17]:
model.evaluate(tf_test_set)

250/250 [==============================] - 16s 64ms/step - loss: 0.2781 - accuracy: 0.9260


[0.2781273126602173, 0.9259999990463257]

**Making Predictions and checking metrics**

In [18]:
validation_preds  = model.predict(tf_validation_dataset)["logits"]

probabilities = tf.nn.softmax(validation_preds)
validation_class_preds = np.argmax(probabilities, axis=1)

validation_class_preds


250/250 [==============================] - 14s 42ms/step


array([1, 5, 4, ..., 1, 1, 4])

In [20]:
print(confusion_matrix(validation_class_preds, emotions_data["validation"]["label"]))

print(classification_report(validation_class_preds, emotions_data["validation"]["label"]))

[[149 196  58  84  49  16]
 [191 252  53  90  80  32]
 [ 55  61  19  22  23   8]
 [ 78  94  23  36  34  10]
 [ 52  77  20  30  17  14]
 [ 25  24   5  13   9   1]]
              precision    recall  f1-score   support

           0       0.27      0.27      0.27       552
           1       0.36      0.36      0.36       698
           2       0.11      0.10      0.10       188
           3       0.13      0.13      0.13       275
           4       0.08      0.08      0.08       210
           5       0.01      0.01      0.01        77

    accuracy                           0.24      2000
   macro avg       0.16      0.16      0.16      2000
weighted avg       0.24      0.24      0.24      2000

